In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

#A feedforward network has a structured layout with layers. Each layer consists of a certain number of neurons,
# and each neuron in one layer is connected to all neurons in the next layer

# We will simulate the activation of neurons by selecting a neuron randomly and then propagating this activation
# forward, to mimic the feedforward process

class SimpleFNNModel:
    def __init__(self, layer_sizes=[5, 3, 2]):
        self.layer_sizes = layer_sizes
        self.activations = [[False for _ in range(size)] for size in layer_sizes]

    def activate_neurons(self):
        # Randomly activate neurons in the input layer
        self.activations[0][np.random.randint(0, self.layer_sizes[0])] = True

        # Propagate activation to subsequent layers
        for i in range(1, len(self.layer_sizes)):
            for j in range(self.layer_sizes[i]):
                if any(self.activations[i-1]):
                    self.activations[i][j] = True

    def reset_activations(self):
        self.activations = [[False for _ in range(size)] for size in self.layer_sizes]

    def draw_network(self):
        plt.figure(figsize=(10, 6))
        for layer_index, layer_size in enumerate(self.layer_sizes):
            # Calculate positions
            x = np.full(layer_size, layer_index)
            y = np.linspace(0, 1, layer_size)
            colors = ['green' if act else 'red' for act in self.activations[layer_index]]

            # Draw neurons
            plt.scatter(x, y, c=colors, s=100, label=f'Layer {layer_index}')

            # Draw connections
            if layer_index > 0:
                prev_layer_size = self.layer_sizes[layer_index - 1]
                for prev_neuron_index in range(prev_layer_size):
                    for neuron_index in range(layer_size):
                        plt.plot([layer_index - 1, layer_index], [np.linspace(0, 1, prev_layer_size)[prev_neuron_index], y[neuron_index]], 'gray')

        plt.legend()
        plt.axis('off')
        plt.show()

# Create FNN model instance
fnn_model = SimpleFNNModel()

# Define UI elements
activate_button = widgets.Button(description='Activate Neurons')
reset_button = widgets.Button(description='Reset')
output_area = widgets.Output()

# Define button click actions
def on_activate_clicked(b):
    with output_area:
        output_area.clear_output(wait=True)
        fnn_model.activate_neurons()
        fnn_model.draw_network()

def on_reset_clicked(b):
    with output_area:
        output_area.clear_output(wait=True)
        fnn_model.reset_activations()
        fnn_model.draw_network()

# Set up button callbacks
activate_button.on_click(on_activate_clicked)
reset_button.on_click(on_reset_clicked)

# Display UI
display(widgets.VBox([activate_button, reset_button, output_area]))

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import networkx as nx
import random

class SimpleGNWModel:
    def __init__(self, num_nodes=5):
        self.num_nodes = num_nodes
        self.network = nx.erdos_renyi_graph(n=num_nodes, p=0.5)
        self.activations = {node: False for node in self.network.nodes}

    def activate_node(self):
        selected_node = random.choice(list(self.network.nodes))
        self.activations[selected_node] = True

        # Simulate global broadcast
        for neighbor in self.network.neighbors(selected_node):
            self.activations[neighbor] = True

    def reset_activations(self):
        self.activations = {node: False for node in self.network.nodes}

    def draw_network(self):
        color_map = ['green' if self.activations[node] else 'red' for node in self.network.nodes]
        nx.draw(self.network, node_color=color_map, with_labels=True, node_size=700)
        plt.show()

# Create a GNW model instance
gnw_model = SimpleGNWModel()

# Button to activate a node
activate_button = widgets.Button(description='Activate Node')

# Button to reset activations
reset_button = widgets.Button(description='Reset')

# Output area for the network graph
output_area = widgets.Output()

def on_activate_clicked(b):
    with output_area:
        output_area.clear_output(wait=True)
        gnw_model.activate_node()
        gnw_model.draw_network()

def on_reset_clicked(b):
    with output_area:
        output_area.clear_output(wait=True)
        gnw_model.reset_activations()
        gnw_model.draw_network()

activate_button.on_click(on_activate_clicked)
reset_button.on_click(on_reset_clicked)

display(widgets.VBox([activate_button, reset_button, output_area]))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Set fixed random seed
torch.manual_seed(0)
np.random.seed(0)

# Generate a synthetic classification dataset with 5000 samples, 20 features, 2 classes, and 15 informative features
X, y = make_classification(n_samples=5000, n_features=20, n_classes=2, n_informative=17, random_state=42)
# Split the dataset into training and testing sets with 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

noise_factor = 0  # Define the magnitude of the noise
noise = np.random.randn(*X_train.shape) * noise_factor  # Generate Gaussian noise
X_train = X_train + noise  # Add noise to the training data

# Initialize the standard scaler
scaler = StandardScaler()
# Fit the scaler on the training data and transform it
X_train = scaler.fit_transform(X_train)
# Transform the test data with the same scaler
X_test = scaler.fit_transform(X_test)

# Convert arrays into PyTorch tensors for training and testing sets
X_train_torch = torch.FloatTensor(X_train)
X_test_torch = torch.FloatTensor(X_test)
y_train_torch = torch.LongTensor(y_train)
y_test_torch = torch.LongTensor(y_test)

# Create TensorDataset from tensors
train_dataset = TensorDataset(X_train_torch, y_train_torch)
test_dataset = TensorDataset(X_test_torch, y_test_torch)

# Create DataLoader for both training and testing datasets to iterate over batches
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

class FeedforwardNet(nn.Module):
    def __init__(self):
        super(FeedforwardNet, self).__init__()
        self.fc1 = nn.Linear(20, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class GNWNet(nn.Module):
    def __init__(self):
        super(GNWNet, self).__init__()
        self.fc1 = nn.Linear(20, 64)
        self.attention = nn.Linear(64, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.fc1(x)
        attention_weights = torch.sigmoid(self.attention(x))
        x = x * attention_weights
        x = self.relu(x)
        x = self.fc2(x)
        return x

def train_model(model, train_loader, criterion, optimizer):
    model.train()
    total_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    average_loss = total_loss / len(train_loader)
    return average_loss

def evaluate_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    average_loss = total_loss / len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)
    return average_loss, accuracy

# Lists to store
losses_basic_train = []
losses_gnw_train = []
losses_basic_val = []
losses_gnw_val = []
accuracy_basic_net = []
accuracy_gnw_net = []

basic_net = FeedforwardNet()
gnw_net = GNWNet()
criterion = nn.CrossEntropyLoss()
optimizer_basic = optim.Adam(basic_net.parameters(), lr=0.00001)
optimizer_gnw = optim.Adam(gnw_net.parameters(), lr=0.00001)

patience = 10  # Number of epochs to wait
best_val_loss_basic = float('inf')  # Initialize the best validation loss for basic_net
epochs_no_improve_basic = 0  # Counter for epochs with no improvement for basic_net

# Training and validation for FeedforwardNet
for epoch in range(3000):
    train_loss = train_model(basic_net, train_loader, criterion, optimizer_basic)
    val_loss, accuracy = evaluate_model(basic_net, test_loader, criterion)
    losses_basic_train.append(train_loss)
    losses_basic_val.append(val_loss)
    accuracy_basic_net.append(accuracy)

    # Check if the validation loss improved
    if val_loss < best_val_loss_basic:
        best_val_loss_basic = val_loss
        epochs_no_improve_basic = 0
    else:
        epochs_no_improve_basic += 1

    # Early stopping check
    if epochs_no_improve_basic >= patience:
        print(f"Early stopping triggered for FeedforwardNet at epoch {epoch}")
        break

best_val_loss_gnw = float('inf')  # Initialize the best validation loss for gnw_net
epochs_no_improve_gnw = 0  # Counter

# Training and validation for GNWNet with early stopping
for epoch in range(1000):
    train_loss = train_model(gnw_net, train_loader, criterion, optimizer_gnw)
    val_loss, accuracy = evaluate_model(gnw_net, test_loader, criterion)
    losses_gnw_train.append(train_loss)
    losses_gnw_val.append(val_loss)
    accuracy_gnw_net.append(accuracy)

    # Check if the validation loss improved
    if val_loss < best_val_loss_gnw:
        best_val_loss_gnw = val_loss
        epochs_no_improve_gnw = 0
    else:
        epochs_no_improve_gnw += 1

    # Early stopping check
    if epochs_no_improve_gnw >= patience:
        print(f"Early stopping triggered for GNWNet at epoch {epoch}")
        break

# Plot for FeedforwardNet
plt.figure(figsize=(10, 6))
plt.plot(losses_basic_train, label='Training Loss')
plt.plot(losses_basic_val, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('FeedforwardNetwork Training vs Validation Loss')
plt.legend()
plt.show()

# Plot for GNWNet
plt.figure(figsize=(10, 6))
plt.plot(losses_gnw_train, label='Training Loss')
plt.plot(losses_gnw_val, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('GNWNetwork Training vs Validation Loss')
plt.legend()
plt.show()